In [27]:
import pandas as pd
import numpy as np
from datetime import datetime

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor

In [2]:
links = pd.read_csv('E:\\Python\\RSML-2\\RSML_HW1\\links.csv')
movies = pd.read_csv('E:\\Python\\RSML-2\\RSML_HW1\\movies.csv')
ratings = pd.read_csv('E:\\Python\\RSML-2\\RSML_HW1\\ratings.csv')
tags = pd.read_csv('E:\\Python\\RSML-2\\RSML_HW1\\tags.csv')

In [3]:
tags.head(20)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
5,2,89774,Tom Hardy,1445715205
6,2,106782,drugs,1445715054
7,2,106782,Leonardo DiCaprio,1445715051
8,2,106782,Martin Scorsese,1445715056
9,7,48516,way too long,1169687325


In [30]:
set(tags.tag)

{'Post apocalyptic',
 'Medieval',
 'too long',
 'cattle drive',
 'free to download',
 'stephen king',
 'Seth MacFarlane',
 'Ben Affleck',
 'Arthur C. Clarke',
 'Henry Darger',
 'claustrophobic',
 'joss whedon',
 'computer',
 'food',
 'elegiac',
 'child abuse',
 'ogres',
 'E.M. Forster',
 'rape',
 'blood splatters',
 '2001-like',
 'convent',
 'Australia',
 'narrated',
 'Canada',
 'wine',
 'Dodie Smith',
 'C.S. Lewis',
 'cross dressing',
 'abstract',
 'assassination',
 'hotel',
 'scandal',
 'stylish',
 'Kevin Costner',
 'mel gibson',
 'best comedy',
 'Sundance award winner',
 'Jack Nicholson',
 'depression',
 'Favelas',
 'homosexuality',
 'Rogers and Hammerstein',
 'wry',
 'Insane',
 'immigration',
 'Stand Up',
 'John Travolta',
 'Horrid characterisation',
 'tobacco',
 'unnecessary sequel',
 'CGI',
 'sad',
 'plot twist',
 'Simon Pegg',
 'Hollywood',
 'Russell Crowe',
 'Eric Bana',
 'visual',
 'wedding',
 'loneliness',
 'rebellion',
 'Orlando Bloom',
 'achronological',
 'no dialogue',
 'i

In [4]:
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [5]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-','').split('|'))

In [6]:
movie_genres = [change_string(g) for g in movies.genres.values]

In [7]:
movie_genres[:20]

['Adventure Animation Children Comedy Fantasy',
 'Adventure Children Fantasy',
 'Comedy Romance',
 'Comedy Drama Romance',
 'Comedy',
 'Action Crime Thriller',
 'Comedy Romance',
 'Adventure Children',
 'Action',
 'Action Adventure Thriller',
 'Comedy Drama Romance',
 'Comedy Horror',
 'Adventure Animation Children',
 'Drama',
 'Action Adventure Romance',
 'Crime Drama',
 'Drama Romance',
 'Comedy',
 'Comedy',
 'Action Comedy Crime Drama Thriller']

In [8]:
# добавим тэги
movies_with_tags = movies.join(tags.set_index('movieId'), on='movieId')

In [9]:
movies_with_tags.head()

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09


In [10]:
movies_with_tags.tag.unique()

array(['pixar', 'fun', 'fantasy', ..., 'star wars', 'gintama', 'remaster'],
      dtype=object)

In [11]:
movies_with_tags.dropna(inplace=True)

In [12]:
tag_strings = []
movies2 = []

for movie, group in tqdm(movies_with_tags.groupby('title')):
    tag_strings.append(' '.join([str(s).replace(' ', '').replace('-', '') for s in group.tag.values]))
    movies2.append(movie)

In [78]:
movies_tags = pd.DataFrame(list(zip(movies2,tag_strings)), columns=['title', 'tag'])
movies_genres_tags = movies.join(movies_tags.set_index('title'), on='title')
movies_genres_tags['genres'] = movies_genres_tags['genres'].apply(change_string)
movies_genres_tags = movies_genres_tags.dropna()
movies_genres_tags.head()

,movieId,title,genres,tag
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,pixar pixar fun
1,2,Jumanji (1995),Adventure Children Fantasy,fantasy magicboardgame RobinWilliams game
2,3,Grumpier Old Men (1995),Comedy Romance,moldy old
4,5,Father of the Bride Part II (1995),Comedy,pregnancy remake
6,7,Sabrina (1995),Comedy Romance,remake


# Строим рекомедацию на жанрах

In [14]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movies_genres_tags.genres.values)

In [15]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [16]:
neig_reg = KNeighborsRegressor(n_neighbors=10, n_jobs=-1, metric='manhattan')
neig_reg.fit(X_train_counts, X_train_tfidf)

KNeighborsRegressor(metric='manhattan', n_jobs=-1, n_neighbors=10)

In [17]:
test = change_string('Adventure|Comedy|Fantasy|Crime')

predict = count_vect.transform([test])
X_tfidf = tfidf_transformer.transform(predict)

predicted_movies = neig_reg.kneighbors(X_tfidf, return_distance=True)
predicted_movies

(array([[1.76082037, 1.76082037, 1.76082037, 1.76082037, 1.76082037,
         1.76542145, 1.96404785, 2.05017702, 2.05017702, 2.05017702]]),
 array([[ 624, 1516,  939,  875,  945, 1262,  217,  653, 1296,  275]],
       dtype=int64))

# Строим рекомедацию на тегах

In [18]:
movies_genres_tags.tag

0                                    pixar pixar fun
1          fantasy magicboardgame RobinWilliams game
2                                          moldy old
4                                   pregnancy remake
6                                             remake
                            ...                     
9681                      Comedy funny RachelMcAdams
9692    adventure AliciaVikander videogameadaptation
9709                 JoshBrolin RyanReynolds sarcasm
9710                           EmiliaClarke starwars
9732                   anime comedy gintama remaster
Name: tag, Length: 1574, dtype: object

In [86]:
mg_tags2 = movies_genres_tags['tag'].value_counts()

In [87]:
mg_tags2 = mg_tags2.reset_index()

In [88]:
mg_tags2.columns = ['tag', 'count']
mg_tags2

,tag,count
0,InNetflixqueue,109
1,Disney,15
2,aliens,10
3,Shakespeare,10
4,religion,9
...,...,...
1031,moldy old,1
1032,Disney race,1
1033,aviation spoof,1
1034,birds parrots,1


In [91]:
movies_genres_tags2 =  movies_genres_tags.join(mg_tags2.set_index('tag'), on='tag')

In [92]:
movies_genres_tags2

,movieId,title,genres,tag,count
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,pixar pixar fun,1
1,2,Jumanji (1995),Adventure Children Fantasy,fantasy magicboardgame RobinWilliams game,1
2,3,Grumpier Old Men (1995),Comedy Romance,moldy old,1
4,5,Father of the Bride Part II (1995),Comedy,pregnancy remake,1
6,7,Sabrina (1995),Comedy Romance,remake,3
...,...,...,...,...,...
9681,183611,Game Night (2018),Action Comedy Crime Horror,Comedy funny RachelMcAdams,1
9692,184471,Tomb Raider (2018),Action Adventure Fantasy,adventure AliciaVikander videogameadaptation,1
9709,187593,Deadpool 2 (2018),Action Comedy SciFi,JoshBrolin RyanReynolds sarcasm,1
9710,187595,Solo: A Star Wars Story (2018),Action Adventure Children SciFi,EmiliaClarke starwars,1


In [93]:
movies_genres_tags2.loc[movies_genres_tags2['count'] < 5, ['tag']] = ['other']
movies_genres_tags2

,movieId,title,genres,tag,count
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,other,1
1,2,Jumanji (1995),Adventure Children Fantasy,other,1
2,3,Grumpier Old Men (1995),Comedy Romance,other,1
4,5,Father of the Bride Part II (1995),Comedy,other,1
6,7,Sabrina (1995),Comedy Romance,other,3
...,...,...,...,...,...
9681,183611,Game Night (2018),Action Comedy Crime Horror,other,1
9692,184471,Tomb Raider (2018),Action Adventure Fantasy,other,1
9709,187593,Deadpool 2 (2018),Action Comedy SciFi,other,1
9710,187595,Solo: A Star Wars Story (2018),Action Adventure Children SciFi,other,1


In [94]:
count_vect_tags = CountVectorizer()
X_train_counts2 = count_vect_tags.fit_transform(movies_genres_tags2.tag.values)

In [95]:
tfidf_tags_transformer = TfidfTransformer()
X_train_tfidf2 = tfidf_tags_transformer.fit_transform(X_train_counts2)

In [96]:
neig_reg_tags = KNeighborsRegressor(n_neighbors=10, n_jobs=-1, metric='manhattan')
neig_reg_tags.fit(X_train_counts2, X_train_tfidf2)

KNeighborsRegressor(metric='manhattan', n_jobs=-1, n_neighbors=10)

In [97]:
test_tags = 'mafia funny L.A.'

predict_tags = count_vect_tags.transform([test_tags])
X_tfidf2_tags = tfidf_tags_transformer.transform(predict_tags)

res_tags = neig_reg_tags.kneighbors(X_tfidf2_tags, return_distance=True)
res_tags

(array([[0., 0., 0., 0., 0., 0., 0., 2., 2., 2.]]),
 array([[ 526,  528,  398,  301,  180,  322,    7, 1053, 1052, 1054]],
       dtype=int64))

In [98]:
tfidf_transformer3 = TfidfVectorizer()
tfidf_transformer3.fit(movies_genres_tags['genres'])

TfidfVectorizer()

In [100]:
with_ratings = movies_genres_tags2.merge(ratings)

In [101]:
for_model_df = with_ratings[with_ratings['userId']==75]
for_model_df

,movieId,title,genres,tag,count,userId,rating,timestamp
1597,47,Seven (a.k.a. Se7en) (1995),Mystery Thriller,other,1,75,2.0,1158968272
3859,260,Star Wars: Episode IV - A New Hope (1977),Action Adventure SciFi,other,1,75,5.0,1158989756
8265,515,"Remains of the Day, The (1993)",Drama Romance,other,1,75,2.0,1158989888
8809,541,Blade Runner (1982),Action SciFi Thriller,other,1,75,4.0,1158989785
9585,589,Terminator 2: Judgment Day (1991),Action SciFi,other,1,75,4.5,1158989827
9970,592,Batman (1989),Action Crime Thriller,superhero,8,75,3.5,1158989896
12754,903,Vertigo (1958),Drama Mystery Romance Thriller,other,1,75,1.0,1158968143
12920,908,North by Northwest (1959),Action Adventure Mystery Romance Thriller,other,1,75,4.5,1158967045
13075,912,Casablanca (1942),Drama Romance,other,1,75,4.0,1158967840
13405,920,Gone with the Wind (1939),Drama Romance War,other,4,75,3.0,1158967791


In [102]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [121]:
X, y = tfidf_transformer3.transform(for_model_df['genres']), for_model_df['rating']

In [122]:
from sklearn.model_selection import train_test_split

In [123]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

In [124]:
lr.fit(X_train, y_train)

LinearRegression()

In [125]:
predicted = lr.predict(X_test)

In [126]:
for_model_df

,movieId,title,genres,tag,count,userId,rating,timestamp
1597,47,Seven (a.k.a. Se7en) (1995),Mystery Thriller,other,1,75,2.0,1158968272
3859,260,Star Wars: Episode IV - A New Hope (1977),Action Adventure SciFi,other,1,75,5.0,1158989756
8265,515,"Remains of the Day, The (1993)",Drama Romance,other,1,75,2.0,1158989888
8809,541,Blade Runner (1982),Action SciFi Thriller,other,1,75,4.0,1158989785
9585,589,Terminator 2: Judgment Day (1991),Action SciFi,other,1,75,4.5,1158989827
9970,592,Batman (1989),Action Crime Thriller,superhero,8,75,3.5,1158989896
12754,903,Vertigo (1958),Drama Mystery Romance Thriller,other,1,75,1.0,1158968143
12920,908,North by Northwest (1959),Action Adventure Mystery Romance Thriller,other,1,75,4.5,1158967045
13075,912,Casablanca (1942),Drama Romance,other,1,75,4.0,1158967840
13405,920,Gone with the Wind (1939),Drama Romance War,other,4,75,3.0,1158967791


In [127]:
y_test

40543    4.0
12920    4.5
16593    3.5
27376    1.0
34033    4.0
39171    1.0
19155    4.0
27264    1.5
9970     3.5
30972    1.0
Name: rating, dtype: float64

# Оценить RMSE на тестовой выборке

In [128]:
from sklearn.metrics import mean_squared_error

In [129]:
mean_squared_error(y_test, predicted)

3.1968232562769856